In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Data Ingest and Cleaning

In [2]:
#Data Ingest and Cleaning
#read general sales file
#set data types and clean 
Omphalos_Sales = pd.read_csv("Omphalos_Sales_2017-2018.csv", encoding = "ISO-8859-1")

dropInd=Omphalos_Sales[Omphalos_Sales['UPC']=='Filler'].index
Omphalos_Sales.drop(index = dropInd, inplace=True)
Omphalos_Sales["UPC"] = Omphalos_Sales["UPC"].astype(float).astype(int)
#Omphalos_Sales["UPC"] = Omphalos_Sales["UPC"].astype(np.int64)
Omphalos_Sales["Wk End"] = pd.to_datetime(Omphalos_Sales['Wk End'], infer_datetime_format=True)

Omphalos_Sales["YEAR"] = Omphalos_Sales["Wk End"].dt.year
Omphalos_Sales["WeekNum"] = Omphalos_Sales["Wk End"].dt.week
Omphalos_Sales["YEARWeekNum"] = Omphalos_Sales["YEAR"].astype(str)+"-"+Omphalos_Sales["WeekNum"].astype(str)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Use lighter dataframe and fill na
OSalesLt = Omphalos_Sales[["UPC","Gross Sales","YEAR", "WeekNum", "YEARWeekNum"]].copy()
OSalesLt.fillna(0, inplace = True)

#check that there are no 0 values
OSalesLt.isnull().sum()

UPC            0
Gross Sales    0
YEAR           0
WeekNum        0
YEARWeekNum    0
dtype: int64

In [4]:
#Ingest advertistement campaign data
OmphalosCamp = pd.read_csv("Omphalos_PM_Tonic_Wrap_Up_Report_4.1.19.csv", encoding = "ISO-8859-1")

OmphalosCamp.rename(columns = {' Spend ': 'Spend'}, inplace=True)
OmphalosCamp["UPC"] = OmphalosCamp["UPC"].astype(np.int64)
OmphalosCamp["Date"] = pd.to_datetime(OmphalosCamp['Date'], infer_datetime_format=True)

OmphalosCamp["YEAR"] = OmphalosCamp["Date"].dt.year
OmphalosCamp["WeekNum"] = OmphalosCamp["Date"].dt.week
OmphalosCamp["YEARWeekNum"] = OmphalosCamp["YEAR"].astype(str)+"-"+OmphalosCamp["WeekNum"].astype(str)
OmphalosCampLt = OmphalosCamp.drop(columns=['Account Id', 'Account Name', 'Campaign Id', 'Campaign Name', 'Date',
                        'TCIN Name', 'Impressions', 'Clicks']).copy()

In [5]:
#createvariables for campaign year and campaign weeks
campYear = 2018
campWeeks = np.sort(OmphalosCamp["WeekNum"].unique())

In [6]:
'''
##Additional Cleaning ##

#since there are some UPC with multiple TCINs, merge those spends for the single UPC

UPCtoTCIN = OmphalosCamp.groupby(["UPC","TCIN"])["Account Id"].count().reset_index().groupby(["UPC"])["TCIN"].count()

# spot check that the 1  UPC to many TCIN cases
UPCtoTCIN[UPCtoTCIN>1]


# check one UPC and the multiple TCINs that is linked to it
print(OmphalosCamp[OmphalosCamp["UPC"]==818094000000.0]["TCIN"].unique())

# check if the two products are similar
OmphalosCamp[OmphalosCamp["TCIN"]==12958487].head(1) #the first two TCINs for UPC 818094000000

OmphalosCamp[OmphalosCamp["TCIN"]==14771376].head(1) #there are similar product but slightly different

'''

'\n##Additional Cleaning ##\n\n#since there are some UPC with multiple TCINs, merge those spends for the single UPC\n\nUPCtoTCIN = OmphalosCamp.groupby(["UPC","TCIN"])["Account Id"].count().reset_index().groupby(["UPC"])["TCIN"].count()\n\n# spot check that the 1  UPC to many TCIN cases\nUPCtoTCIN[UPCtoTCIN>1]\n\n\n# check one UPC and the multiple TCINs that is linked to it\nprint(OmphalosCamp[OmphalosCamp["UPC"]==818094000000.0]["TCIN"].unique())\n\n# check if the two products are similar\nOmphalosCamp[OmphalosCamp["TCIN"]==12958487].head(1) #the first two TCINs for UPC 818094000000\n\nOmphalosCamp[OmphalosCamp["TCIN"]==14771376].head(1) #there are similar product but slightly different\n\n'

In [7]:
#roll the Campaign data into weekly periods so it matches with the time series frequency of Omphalos Sales data
#also the one UPC to many TCIN cases,  those spend will be summed to the single UPC

OCWeekly = OmphalosCampLt.groupby(["UPC", "YEAR", "WeekNum", "YEARWeekNum"], as_index=False)["Spend"].sum()

In [8]:
#check OSalesLt for top 3 products for few weeks before and after campaign
top20UPC = [30000064412,12000809941,28400642033,12000504044,30000060834,12000809965,28400639552,30000061190,
30000010402,28400011099,30000169018,52000010978,12000171864,30000562529,30000261927,30000311820,52000010886,
12000809972,52000340242,30000012000]

In [9]:
#check that there is sales for top 20 UPCS in 2018 for weeks 18 10 26 (2 months before campaign)
OSalesLt[OSalesLt["UPC"].isin(top20UPC) & OSalesLt["YEAR"].isin([2018]) & OSalesLt["WeekNum"].isin(np.arange(18,27)) ]\
        .groupby("UPC").agg({"Gross Sales": "sum"})
#confirmed that's true

,Gross Sales
UPC,
12000171864,2590.91
12000504044,6980.94
12000809941,4710.45
12000809965,3500.91
12000809972,2011.97
28400011099,5262.54
28400639552,8056.15
28400642033,7916.42
30000010402,6344.40


In [10]:
#check that there is sales for top 20UPCS in 2018 for weeks 45 to 52 (2 months after campaign)
OSalesLt[OSalesLt["UPC"].isin(top20UPC) & OSalesLt["YEAR"].isin([2018]) & OSalesLt["WeekNum"].isin(np.arange(45,53)) ]\
        .groupby("UPC").agg({"Gross Sales": "sum"})
#confirmed that most have sales, except 2 UPCs

,Gross Sales
UPC,
12000171864,11034.50
12000504044,13403.87
12000809941,18840.16
12000809965,14432.91
12000809972,9249.17
28400011099,7744.84
28400639552,9361.11
28400642033,12727.33
30000010402,10180.76


In [11]:
#For the linear regression, we inclued all weeks 18 to 52 ("sandwich" campaign weeks 27 - 44)
weeksList = np.arange(18,53)
weeksList

array([18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52])

In [12]:
campWeeks #these are the weeks when campaign was active

array([27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44])

In [13]:
#make YearWkNumdf

YearWkNum18 = list(map(lambda x: "2018-"+str(x),weeksList))
YearWkNumdf = pd.DataFrame({"YEAR": [2018]*35, "WeekNum": list(weeksList),
                            "YEARWeekNum":YearWkNum18})

In [14]:
#check that all Omphalos Campaign UPC  is in sales
#there are actually soome Campaign UPCs that are msising from sales UPCs
SalesUPC = set(OSalesLt.UPC.unique())

In [15]:
CampUPC = set(OCWeekly.UPC.unique())

In [16]:
CampUPC - SalesUPC 
#these are the UPCs that are missing from Sales file 
#check if they have large revenue later

{28400154406,
 28400154437,
 28400154550,
 28400154659,
 28400155861,
 28400655927,
 30000018804,
 493000000000}

In [17]:
joinUPC = CampUPC & SalesUPC #there are 163 UPCs in both files
print(type(joinUPC))
print(len(joinUPC))

<class 'set'>
165


In [18]:
#checkpoint copy DF
#OCWeeklyold = OCWeekly.copy() #for checkpoint

In [19]:
#make sure each UPC has rows for each Week in weekList
#fill in the weeks that have no data with 0 sales and 0 spend
def mergeYWN(df):
    #each df is of a single UPC df
    df2 = df.merge(YearWkNumdf, how = "right", left_on="YEARWeekNum", right_on="YEARWeekNum")
    return df2

In [20]:
SalesSlicedf = OSalesLt.groupby(["UPC"]).apply(mergeYWN)

In [21]:
SalesSlicedf.shape #check that shape is correct, should be OSalesLt unique UPC * num weeks in YearWeekNumdf

(38462, 7)

In [22]:
#fill in the missing rows
SalesSlicedf.drop(columns = ["UPC","YEAR_x", "WeekNum_x"], inplace = True) #drop the extra UPC columns that has null values
SalesSlicedf.reset_index(inplace = True) #reset index so true UPC is acolumns
SalesSlicedf.drop(columns = ["level_1"], inplace = True) #drop extra column

#fillnas Sales with 0
SalesSlicedf["Gross Sales"].fillna(0, inplace = True)


SalesSlicedf.rename(columns={"YEAR_y": "YEAR", "WeekNum_y": "WeekNum"}, inplace=True)

In [23]:
#merge OCWeekly with all the YEARWeekNumdf
OCWeeklyFull = OCWeekly.groupby(["UPC"]).apply(mergeYWN)

#do the same reorg as SalesSlicedf
OCWeeklyFull.drop(columns = ["UPC","YEAR_x", "WeekNum_x"], inplace = True) #drop the extra UPC columns that has null values
OCWeeklyFull.reset_index(inplace = True) #reset index so true UPC is acolumns
OCWeeklyFull.drop(columns = ["level_1"], inplace = True) #drop extra column
OCWeeklyFull["Spend"].fillna(0, inplace = True)

OCWeeklyFull.rename(columns={"YEAR_y": "YEAR", "WeekNum_y": "WeekNum"}, inplace=True)

In [24]:
#merge campaign and sales data for week 18 to 52 
Omerge = OCWeeklyFull.merge(SalesSlicedf, how ="inner", left_on = ["UPC", "YEARWeekNum"], right_on = ["UPC", "YEARWeekNum"])

In [25]:
Omerge.drop(columns=["YEAR_x", "WeekNum_x"], inplace=True)

Omerge.rename(columns={"YEAR_y": "YEAR", "WeekNum_y": "WeekNum"}, inplace=True)

Analysis

In [26]:
'''
##Side-by-side Analysis##
#for side by side analysis between 2017 and 2018 campsign weeks. This approach was 
#found unviable later because of the sparisty of sales in 2017

def sidetoside(df):
    #each df is by individual UPC
    df2017 = df[df["YEAR"].isin([2017])].copy()
    df2017.rename(columns={"Spend": "2017Spend", "Gross Sales": "2017 Sales"}, inplace= True)
    df2018= df[df["YEAR"].isin([2018])].copy()
    df2018.rename(columns={"Spend": "2018Spend", "Gross Sales": "2018 Sales"}, inplace= True)
    result = df2017.merge(df2018, how="inner", left_on="WeekNum", right_on="WeekNum")
    return result


TS2S = Omerge.groupby(["UPC"]).apply(sidetoside)
TS2S.reset_index(inplace=True)

TS2S.drop(columns=['level_1', 'UPC_x','YEARWeekNum_x','YEAR_x','UPC_y', 'YEARWeekNum_y','YEAR_y'], inplace=True)
TS2S.to_csv("TS2S.csv")
TS2StoOSales = TS2S.groupby(["UPC"], as_index=False)["2018 Sales"].sum()

TS2StoOSales.sort_values(by = ["2018 Sales"], ascending=False, inplace=True)

top20UPC = list(TS2StoOSales["UPC"][:20])

TS2Stop = TS2S[TS2S["UPC"].isin(top20UPC)]


##output to csv
#TS2Stop.to_csv("TS2Stop.csv")

'''


'\n##Side-by-side Analysis##\n#for side by side analysis between 2017 and 2018 campsign weeks. This approach was \n#found unviable later because of the sparisty of sales in 2017\n\ndef sidetoside(df):\n    #each df is by individual UPC\n    df2017 = df[df["YEAR"].isin([2017])].copy()\n    df2017.rename(columns={"Spend": "2017Spend", "Gross Sales": "2017 Sales"}, inplace= True)\n    df2018= df[df["YEAR"].isin([2018])].copy()\n    df2018.rename(columns={"Spend": "2018Spend", "Gross Sales": "2018 Sales"}, inplace= True)\n    result = df2017.merge(df2018, how="inner", left_on="WeekNum", right_on="WeekNum")\n    return result\n\n\nTS2S = Omerge.groupby(["UPC"]).apply(sidetoside)\nTS2S.reset_index(inplace=True)\n\nTS2S.drop(columns=[\'level_1\', \'UPC_x\',\'YEARWeekNum_x\',\'YEAR_x\',\'UPC_y\', \'YEARWeekNum_y\',\'YEAR_y\'], inplace=True)\nTS2S.to_csv("TS2S.csv")\nTS2StoOSales = TS2S.groupby(["UPC"], as_index=False)["2018 Sales"].sum()\n\nTS2StoOSales.sort_values(by = ["2018 Sales"], ascendi

In [27]:
'''
##Iterate to Same-Year Analysis##
Checking delta Sales vs. campaign spend for the weeks that the campaign was on

df2018 = Omerge2018.groupby(["UPC", "Spend>0"], as_index=False).agg({"WeekNum":"count", "Spend": "sum", "Gross Sales": "sum"})
df2018.rename(columns={"WeekNum": "WeekNumCnt", "Spend": "totSpend", "Gross Sales": "toOSales"}, inplace=True)

df2018["weeklySales"]=df2018["toOSales"]/df2018["WeekNumCnt"]
df2018["weeklySpend"]=df2018["totSpend"]/df2018["WeekNumCnt"]

#first check if there are any no spend that did not have any sales
df2018[df2018["toOSales"]==0]

df2018zeroSales = df2018[df2018["toOSales"]==0].copy() #these will be removed from the table

df2018.drop(df2018[df2018["toOSales"]==0].index, inplace=True) #drop the no sales UPCs

checkmiss=df2018.groupby(["UPC"])["Spend>0"].count()<2

oneRowUPC = list(checkmiss[checkmiss].index)

df2018noZeroSales = df2018[df2018["UPC"].isin(oneRowUPC)].copy() 
# these only have weeks that have sales

df2018.drop(df2018[df2018["UPC"].isin(oneRowUPC)].index, inplace=True) #drop the no sales UPCs
checkmiss2=df2018.groupby(["UPC"])["Spend>0"].count()<2
checkmiss2[checkmiss2] #checked that all have been droped
df2018.sort_values(by=["UPC", "Spend>0"], inplace = True)

# #split and merge
df2018left = df2018[df2018["Spend>0"]==0].copy()

df2018left.drop(columns=["Spend>0", "totSpend", "weeklySpend"], inplace = True)

df2018left.rename(columns={"WeekNumCnt": "noSpendWeekCnt", "toOSales":"noSpendtoOSales", 
                    "weeklySales": "noSpendweeklySales"}, inplace = True)

df2018right = df2018[df2018["Spend>0"]==1].copy()

df2018right.rename(columns={"WeekNumCnt": "SpendWeekCnt", "toOSales":"SpendtoOSales", 
                   "weeklySales": "SpendweeklySales"}, inplace = True)
df2018right.drop(columns=["Spend>0"], inplace=True)

df2018M = df2018left.merge(df2018right, how = "outer", left_on=["UPC"], right_on =["UPC"])

df2018M["delWeeklySales"] = df2018M["SpendweeklySales"]-df2018M["noSpendweeklySales"]
df2018M["weeklyROI"] = df2018M["delWeeklySales"]/df2018M["weeklySpend"]

#save this table
#df2018M.to_csv("df2018M.csv")


'''

'\n##Iterate to Same-Year Analysis##\nChecking delta Sales vs. campaign spend for the weeks that the campaign was on\n\ndf2018 = Omerge2018.groupby(["UPC", "Spend>0"], as_index=False).agg({"WeekNum":"count", "Spend": "sum", "Gross Sales": "sum"})\ndf2018.rename(columns={"WeekNum": "WeekNumCnt", "Spend": "totSpend", "Gross Sales": "toOSales"}, inplace=True)\n\ndf2018["weeklySales"]=df2018["toOSales"]/df2018["WeekNumCnt"]\ndf2018["weeklySpend"]=df2018["totSpend"]/df2018["WeekNumCnt"]\n\n#first check if there are any no spend that did not have any sales\ndf2018[df2018["toOSales"]==0]\n\ndf2018zeroSales = df2018[df2018["toOSales"]==0].copy() #these will be removed from the table\n\ndf2018.drop(df2018[df2018["toOSales"]==0].index, inplace=True) #drop the no sales UPCs\n\ncheckmiss=df2018.groupby(["UPC"])["Spend>0"].count()<2\n\noneRowUPC = list(checkmiss[checkmiss].index)\n\ndf2018noZeroSales = df2018[df2018["UPC"].isin(oneRowUPC)].copy() \n# these only have weeks that have sales\n\ndf2018.

Ridge Regression

In [28]:
#use Omerge for machine learning, linear regression to find spend ROI
from sklearn.linear_model import Ridge

#crate a function to faciliate the fitting and graphing functions later
def fitridge(df):
    #each df is by a single UPC
    xfields = ["Spend"]
    yfields = ["Gross Sales"]
    lr = Ridge(1e1).fit(df[xfields], df[yfields])
    return lr.coef_[0][0]

In [29]:
#find ROI by UPC
SpendCoefbyUPC = Omerge.groupby(["UPC"]).apply(fitridge)

CoefbyUPCdf = pd.DataFrame(SpendCoefbyUPC, columns=["SpendCoef"])

toOSales=Omerge.groupby(["UPC"])["Gross Sales"].sum()

#append total sales and brand name
dftoOSales = pd.DataFrame({"toOSalesWk18-52":toOSales})

infobyUPCdf=Omphalos_Sales.groupby(["UPC", "BU", "Brand Name", "UPC Title"], as_index= False)["Gross Sales"].count().copy()

infobyUPCdf.set_index(["UPC"], inplace=True)

infobyUPCdf.drop(columns=["Gross Sales"], inplace = True)

#merge all of them together and sort by total Sales
dfAll = (CoefbyUPCdf.join(dftoOSales, how="left")).join(infobyUPCdf, how="left")
dfAll.sort_values(by=["toOSalesWk18-52"], ascending = False, inplace=True)

#write output to csv file 
dfAll.to_csv("dfAll04082019.csv")



In [30]:
#Omerge only has 2018 data but just make a copy
#Omerge2018 = Omerge[Omerge["YEAR"].isin([2018])].copy()


In [31]:
#merge Omerge with Brand info
OmergeBrand = Omerge.merge(infobyUPCdf, how = "left", left_on =["UPC"], right_on = ["UPC"] )

#group OmergeBrand by 
OmergeBrand2 = OmergeBrand.groupby(["Brand Name", "WeekNum"], as_index = False).agg({"Spend":"sum", "Gross Sales":"sum"})

In [32]:
#find ROI by brand using ridge regression
SpendCoefbyBrand = OmergeBrand2.groupby(["Brand Name"]).apply(fitridge)
CoefbyBranddf = pd.DataFrame(SpendCoefbyBrand, columns=["SpendCoef"])
toOSalesBrand=OmergeBrand2.groupby(["Brand Name"])["Gross Sales"].sum()

#append total sales and brand name
dftoOSalesBrand = pd.DataFrame({"toOSalesWk18-52":toOSalesBrand})
#merge the ROI with the brand information
dfAll = CoefbyBranddf.join(dftoOSalesBrand, how="left")

dfAll.sort_values(by=["toOSalesWk18-52"], ascending=False, inplace = True)

#output to csv
#dfAll.to_csv("dfAllBrand04102019.csv")

In [33]:
#make excel table of ROI results
dfAllBrand = dfAll
dfAllUPC = pd.read_csv("dfAll04082019.csv")
dfAllBrand.reset_index(inplace=True)
#put the data side to side
dfAllUPCBrand = dfAllUPC.merge(dfAllBrand, how = "left", left_on =["Brand Name"], right_on =["Brand Name"]) 

dfAllUPCBrand.rename(columns={"SpendCoef_x":"UPCSpendCoef", "toOSalesWk18-52_x":"UPCtoOSalesWk18-52",
                      "SpendCoef_y":"BrandSpendCoef", "toOSalesWk18-52_y":"BrandtoOSalesWk18-52"}, inplace = True)

#output the appended table to csv
dfAllUPCBrand.to_csv("dfAllUPCBrand04102019.csv")

In [34]:
#charting spend vs. sales
# one set of top sales UPC and Brands
# one set of top spend UPC and Brands
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected = True)
plot = plotly.offline.plot
from plotly.offline import iplot

In [35]:
dfAllUPCBrand.sort_values(by= "BrandSpendCoef", inplace = True, ascending = False)
data = [
    go.Bar(
        x=dfAllUPCBrand['UPC Title'].astype("category"), # assign x as the dataframe column 'UPC'
        y=dfAllUPCBrand['UPCSpendCoef'], #ROI at the product level
        name='Product ROI',
        base = 0,
        width = 0.3
    ),
    go.Bar(
        x=dfAllUPCBrand['UPC Title'].astype("category"),# assign x as the dataframe column 'UPC'
        y=dfAllUPCBrand['BrandSpendCoef'],
        name='Brand ROI',
        base = 0,
        width = 0.5
    )

]

layout = go.Layout(
    barmode='overlay',
    title='ROI between Product vs. Brand'
)

fig = go.Figure(data=data, layout=layout)


iplot(fig, show_link=True)


In [36]:
def plotSpendSale(df, title):
    trace1 = go.Scatter(
    x=dfviz["WeekNum"],
    y=dfviz["Spend"],
    name='Spend')
    
    trace2 = go.Scatter(
    x=dfviz["WeekNum"],
    y=dfviz["Gross Sales"],
    name='Sales',
    yaxis = "y2")

    data = [trace1, trace2]
    
    layout = go.Layout(
    #barmode='group',
    title=title,
    width=800,
    yaxis=dict(
        title='Spend',
        titlefont=dict(
            color='#1f77b4'
        ),
        tickfont=dict(
            color='#1f77b4'
        )
    ),
        
    yaxis2=dict(
        title='Sales',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        
        overlaying='y',
        side='right',
        
        )
    )    

    fig = go.Figure(data=data, layout=layout)
    plot = plotly.offline.plot
    title2 = title.replace(" ", "_").replace(":", "_")
    plot(fig, filename=title2+".html")

In [37]:
top6UPC = OmergeBrand.groupby('UPC')['Gross Sales'].sum().sort_values(ascending=False).index[:6]

#make the charts and save in directory
for UPC in top6UPC:
    dfviz = OmergeBrand.loc[OmergeBrand['UPC'] == UPC]
    dfviz = dfviz.sort_values("WeekNum")
    brand = dfviz["Brand Name"].values[0]
    title = 'UPC:'+str(UPC) +" brand:"+brand
    
    plotSpendSale(df=dfviz, title=title)


top6Brand = dfAllBrand.sort_values(by=["toOSalesWk18-52"], ascending=False)["Brand Name"][:6]

for brand in top6Brand:
    dfviz = OmergeBrand2.loc[OmergeBrand2['Brand Name'] == brand]
    dfviz = dfviz.sort_values("WeekNum")
    title = "brand:"+brand
    
    plotSpendSale(df=dfviz, title=title)


In [38]:
#makecharts for UPCs that were allocated top spend
top6spendUPC = OmergeBrand.groupby('UPC')['Spend'].sum().sort_values(ascending=False).index[:6]

for UPC in top6UPC:
    dfviz = OmergeBrand.loc[OmergeBrand['UPC'] == UPC]
    dfviz = dfviz.sort_values("WeekNum")
    brand = dfviz["Brand Name"].values[0]
    title = 'SpendUPC:'+str(UPC) +" brand:"+brand
    
    plotSpendSale(df=dfviz, title=title)

top6spendbrand = OmergeBrand2.sort_values(by=["Spend"], ascending=False)["Brand Name"][:6]

for brand in top6Brand:
    dfviz = OmergeBrand2.loc[OmergeBrand2['Brand Name'] == brand]
    dfviz = dfviz.sort_values("WeekNum")
    title = "Spendbrand:"+brand
    
    plotSpendSale(df=dfviz, title=title)